In [ ]:
!pip install pytorch-lightning
!pip install tensorboard
!pip install timm

In [4]:
## Standard libraries
import os
import json
import math
import numpy as np

## Imports for plotting
import matplotlib.pyplot as plt
%matplotlib inline
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('svg', 'pdf') # For export
from matplotlib.colors import to_rgb
import matplotlib
matplotlib.rcParams['lines.linewidth'] = 2.0
import seaborn as sns
sns.reset_orig()
sns.set()

## Progress bar
from tqdm.notebook import tqdm

## PyTorch
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as data
import torch.optim as optim
# Torchvision
import torchvision
from torchvision import transforms
# PyTorch Lightning

import pytorch_lightning as pl
from pytorch_lightning.callbacks import LearningRateMonitor, ModelCheckpoint

# Tensorboard extension (for visualization purposes later)
from torch.utils.tensorboard import SummaryWriter
%load_ext tensorboard

# Setting the seed
pl.seed_everything(42)

# Ensure that all operations are deterministic on GPU (if used) for reproducibility
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device("cpu")
print("Device:", device)

/tmp/ipykernel_18370/354987388.py:11: DeprecationWarning: `set_matplotlib_formats` is deprecated since IPython 7.23, directly use `matplotlib_inline.backend_inline.set_matplotlib_formats()`
  set_matplotlib_formats('svg', 'pdf') # For export
Global seed set to 42


The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard
Device: cuda:0


In [5]:
#!pip install pytorch-lightning

In [6]:
from torchvision import datasets, transforms

In [7]:
train_dir = '../AnamolyData/train/images' # load from Kaggle
val_dir= '../AnamolyData/val/images'
transform = transforms.Compose([transforms.Resize((512,512)),
                                transforms.ToTensor(),
                                transforms.Normalize([0.5, 0.5, 0.5], 
                                                            [0.5, 0.5, 0.5])
                               ])
train_dataset = datasets.ImageFolder(train_dir, transform=transform)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=18, shuffle=True,drop_last=True) 
val_dataset = datasets.ImageFolder(val_dir, transform=transform)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=18, shuffle=True,drop_last=True)
def get_train_images(num):
    return torch.stack([train_dataset[i][0] for i in range(num)], dim=0)

In [ ]:
for batch in train_loader:
    print(batch[0].shape)
    print(batch[0].min())
    print(batch[0].max())
    break

In [8]:
import torch
import timm

In [ ]:
model_names = timm.list_models('*vgg*')
model_names

In [ ]:
encoder = timm.create_model('vgg19_bn',features_only=True, pretrained=True)

In [ ]:
o = encoder(torch.randn(2, 3, 512, 512))
for x in o:
    print(x.shape)

In [ ]:
encoder=timm.create_model('repvgg_b3',features_only=True, pretrained=False)

In [ ]:
o = encoder(torch.randn(2, 3, 512, 512))
for x in o:
    print(x.shape)

In [9]:
CHECKPOINT_PATH="anamoly_checkpoints"
os.makedirs(CHECKPOINT_PATH, exist_ok=True)

In [10]:
class Encoder(nn.Module):

    def __init__(self):
        super().__init__()
        self.encoder = timm.create_model('vgg19_bn',features_only=True, pretrained=False)


    def forward(self, x):
        return self.encoder(x)[-1]

In [11]:
class Decoder(nn.Module):

    def __init__(self,
                 num_input_channels : int,
                 c_hid : int,
                 act_fn : object = nn.GELU):
        """
        Inputs:
            - num_input_channels : Number of channels of the image to reconstruct. For CIFAR, this parameter is 3
            - base_channel_size : Number of channels we use in the last convolutional layers. Early layers might use a duplicate of it.
            - latent_dim : Dimensionality of latent representation z
            - act_fn : Activation function used throughout the decoder network
        """
        super().__init__()
        
        inter1=c_hid//2
        inter2=inter1//2
        inter3=inter2//2
        
        self.net = nn.Sequential(
            nn.ConvTranspose2d(c_hid, c_hid, kernel_size=3, output_padding=1, padding=1, stride=2), # 16x16 => 32x32
            act_fn(),
            nn.Conv2d(c_hid, c_hid, kernel_size=3, padding=1),
            act_fn(),
            nn.Conv2d(c_hid, c_hid, kernel_size=3, padding=1),
            act_fn(),
            nn.Conv2d(c_hid, c_hid, kernel_size=3, padding=1),
            act_fn(),
            nn.Conv2d(c_hid, c_hid, kernel_size=3, padding=1),
            act_fn(),
            nn.Conv2d(c_hid, c_hid, kernel_size=3, padding=1),
            act_fn(),
            nn.ConvTranspose2d(c_hid, inter1, kernel_size=3, output_padding=1, padding=1, stride=2), # 32x32 => 64x64
            act_fn(),
            nn.Conv2d(inter1, inter1, kernel_size=3, padding=1),
            act_fn(),
            nn.Conv2d(inter1, inter1, kernel_size=3, padding=1),
            act_fn(),
            nn.Conv2d(inter1, inter1, kernel_size=3, padding=1),
            act_fn(),
            nn.Conv2d(inter1, inter1, kernel_size=3, padding=1),
            act_fn(),
            nn.Conv2d(inter1, inter1, kernel_size=3, padding=1),
            act_fn(),
            nn.Conv2d(inter1, inter1, kernel_size=3, padding=1),
            act_fn(),
            nn.ConvTranspose2d( inter1, inter2, kernel_size=3, output_padding=1, padding=1, stride=2), # 64x64 => 128x128
            act_fn(),
            nn.Conv2d(inter2, inter2, kernel_size=3, padding=1),
            act_fn(),
            nn.Conv2d(inter2, inter2, kernel_size=3, padding=1),
            act_fn(),
            nn.Conv2d(inter2, inter2, kernel_size=3, padding=1),
            act_fn(),
            nn.Conv2d(inter2, inter2, kernel_size=3, padding=1),
            act_fn(),
            nn.Conv2d(inter2, inter2, kernel_size=3, padding=1),
            act_fn(),
            nn.ConvTranspose2d(inter2, inter3, kernel_size=3, output_padding=1, padding=1, stride=2), # 128x128 => 256x256
            act_fn(),
            nn.Conv2d(inter3, inter3, kernel_size=3, padding=1),
            act_fn(),
            nn.Conv2d(inter3, inter3, kernel_size=3, padding=1),
            act_fn(),
            nn.Conv2d(inter3, inter3, kernel_size=3, padding=1),
            act_fn(),
            nn.Conv2d(inter3, inter3, kernel_size=3, padding=1),
            act_fn(),
            nn.Conv2d(inter3, inter3, kernel_size=3, padding=1),
            act_fn(),
            nn.ConvTranspose2d(inter3, num_input_channels, kernel_size=3, output_padding=1, padding=1, stride=2), # 256x256 => 512x512
            act_fn(),
            nn.Conv2d(num_input_channels, num_input_channels, kernel_size=3, padding=1),
            nn.Tanh() # The input images is scaled between -1 and 1, hence the output has to be bounded as well
        )

    def forward(self, x):
        x = self.net(x)
        return x

In [12]:
class Autoencoder(pl.LightningModule):

    def __init__(self,
                 latent_dim: int,
                 encoder_class : object = Encoder,
                 decoder_class : object = Decoder,
                 num_input_channels: int = 3,
                 width:int= 512,
                 height:int= 512):
        super().__init__()
        # Saving hyperparameters of autoencoder
        self.save_hyperparameters()
        # Creating encoder and decoder
        self.encoder = encoder_class()
        self.decoder = decoder_class(num_input_channels, latent_dim)
        self.inception= timm.create_model('vgg19_bn',features_only=True, pretrained=True)
        # Example input array needed for visualizing the graph of the network
        self.example_input_array = torch.zeros(2, num_input_channels, width, height)

    def forward(self, x):
        """
        The forward function takes in an image and returns the reconstructed image
        """
        z = self.encoder(x)
        x_hat = self.decoder(z)
        return x_hat

    def _get_reconstruction_loss(self, batch):
        """
        Given a batch of images, this function returns the reconstruction loss (MSE in our case)
        """
        x, _ = batch # We do not need the labels
        x_hat = self.forward(x)
        loss = F.mse_loss(x, x_hat, reduction="none")
        loss = loss.sum(dim=[1,2,3]).mean(dim=[0])
        return loss
    
    @torch.no_grad()
    def _get_inception(self,x):
        z=self.inception(x)[-1]
        return z
    
    def _get_inception_loss(self,batch):
        x, _ = batch # We do not need the labels
        z_hat = self.encoder(x)
        z=self._get_inception(x)
        loss = F.mse_loss(z, z_hat, reduction="none")
        loss = loss.sum(dim=[1,2,3]).mean(dim=[0])
        return loss

    def configure_optimizers(self):
        optimizer = optim.Adam(self.parameters(), lr=1e-3)
        # Using a scheduler is optional but can be helpful.
        # The scheduler reduces the LR if the validation performance hasn't improved for the last N epochs
        scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer,
                                                         mode='min',
                                                         factor=0.2,
                                                         patience=20,
                                                         min_lr=5e-5)
        return {"optimizer": optimizer, "lr_scheduler": scheduler, "monitor": "val_loss"}

    def training_step(self, batch, batch_idx):
        loss1 = self._get_reconstruction_loss(batch)
        loss2 = self._get_inception_loss(batch)
        loss=loss1+loss2
        self.log('train_loss', loss)
        return loss

    def validation_step(self, batch, batch_idx):
        loss = self._get_reconstruction_loss(batch)
        self.log('val_loss', loss)

    def test_step(self, batch, batch_idx):
        loss = self._get_reconstruction_loss(batch)
        self.log('test_loss', loss)

In [13]:
## Model evaluation
weights="../anamoly_checkpoints/anamoly_road_512/lightning_logs/version_7/checkpoints/epoch=20-step=431046.ckpt"
model = Autoencoder.load_from_checkpoint(weights)

Downloading: "https://download.pytorch.org/models/vgg19_bn-c79401a0.pth" to /home/ec2-user/.cache/torch/hub/checkpoints/vgg19_bn-c79401a0.pth


In [14]:
class GenerateCallback(pl.Callback):

    def __init__(self, input_imgs, every_n_epochs=1):
        super().__init__()
        self.input_imgs = input_imgs # Images to reconstruct during training
        self.every_n_epochs = every_n_epochs # Only save those images every N epochs (otherwise tensorboard gets quite large)

    def on_train_epoch_end(self, trainer, pl_module):
        if trainer.current_epoch % self.every_n_epochs == 0:
            # Reconstruct images
            input_imgs = self.input_imgs.to(pl_module.device)
            with torch.no_grad():
                pl_module.eval()
                reconst_imgs = pl_module(input_imgs)
                pl_module.train()
            # Plot and add to tensorboard
            imgs = torch.stack([input_imgs, reconst_imgs], dim=1).flatten(0,1)
            grid = torchvision.utils.make_grid(imgs, nrow=2, normalize=True, range=(-1,1))
            trainer.logger.experiment.add_image("Reconstructions", grid, global_step=trainer.global_step)

In [15]:
def train_Anamoly(latent_dim):
    # Create a PyTorch Lightning trainer with the generation callback
    trainer = pl.Trainer(default_root_dir=os.path.join(CHECKPOINT_PATH, f"anamoly_road_{latent_dim}"),
                         accelerator="cuda" if str(device).startswith("cuda") else "cpu",
                         devices=1,
                         max_epochs=1000,
                         callbacks=[ModelCheckpoint(save_weights_only=True),
                                    GenerateCallback(get_train_images(8), every_n_epochs=10),
                                    LearningRateMonitor("epoch")])
    trainer.logger._log_graph = True         # If True, we plot the computation graph in tensorboard
    trainer.logger._default_hp_metric = None # Optional logging argument that we don't need

    # Check whether pretrained model exists. If yes, load it and skip training
    pretrained_filename = os.path.join(CHECKPOINT_PATH, f"anamoly_road_{latent_dim}.ckpt")
    if os.path.isfile(pretrained_filename):
        print("Found pretrained model, loading...")
        model = Autoencoder.load_from_checkpoint(pretrained_filename)
    else:
        model = Autoencoder(latent_dim=latent_dim)
        trainer.fit(model, train_loader, val_loader)
    # Test best model on validation and test set
    val_result = trainer.test(model, val_loader, verbose=False)
    #test_result = trainer.test(model, test_loader, verbose=False)
    result = {"val": val_result}
    return model, result

In [ ]:
train_Anamoly(latent_dim=512)

In [16]:
from utils import *

In [17]:
def estimate_gaussian(X): 
    """
    Calculates mean and variance of all features 
    in the dataset
    
    Args:
        X (ndarray): (m, n) Data matrix
    
    Returns:
        mu (ndarray): (n,) Mean of all features
        var (ndarray): (n,) Variance of all features
    """

    m, n = X.shape
    mu= np.mean(X,axis=0)
    var= np.var(X,axis=0) 
    return mu, var

In [ ]:
## Train features for checking 
train_features=[]
for i,item in enumerate(tqdm(train_loader)):
    x,y=item[0],item[1]
    model.cuda()
    model.eval()
    x=x.cuda()
    #x=x.unsqueeze(0)
    y=model.encoder(x).detach().cpu()
    train_features.append(y.flatten(start_dim=1).numpy())

  0%|          | 0/6842 [00:00<?, ?it/s]

In [ ]:
train_features=np.concatenate(train_features)

In [ ]:
with open('train_features.npy', 'wb') as f:
    np.save(f, train_features)

In [ ]:
mu, var = estimate_gaussian(train_features)  

In [ ]:
## Normal distribution should look like this
p_train = multivariate_gaussian(train_features, mu, var)

In [ ]:
## Val features and labels for picking the best threshold
val_features=[]
y_val=[]
for i,item in enumerate(tqdm(val_loader)):
    x,y=item[0],item[1]
    model.cuda()
    model.eval()
    x=x.cuda()
    #x=x.unsqueeze(0)
    y=model.encoder(x).detach().cpu()
    val_features.append(y.flatten(start_dim=1).numpy())
    y_val.append(y_val.detch().cpu().numpy())
val_features=np.concatenate(val_features)
y_val=np.concatenate(y_val)

In [ ]:
## Data Out of distribution looks like this
p_val = multivariate_gaussian(val_features, mu, var)

In [ ]:
epsilon, F1 = select_threshold(y_val, p_val)
print('Best epsilon found using cross-validation: %e' % epsilon)
print('Best F1 on Cross Validation Set: %f' % F1)